## 🔍 Importação das Bibliotecas e Leitura do Arquivo CSV
Neste passo, vamos importar as bibliotecas necessárias e carregar o arquivo CSV para análise.



In [32]:
import pandas as pd
import numpy as np
# Caminho do arquivo CSV
caminho_arquivo = 'vendas_modificado.csv'

# Leitura do CSV com fallback de encoding corrigido via bytes
df = pd.read_csv(caminho_arquivo, encoding='latin1')


# Correção de textos com possível corrupção de UTF-8
def corrigir_encoding(texto):
    try:
        return texto.encode('latin1').decode('utf-8')
    except:
        return texto


# Aplicar em campos textuais princicep
for coluna in ['cliente', 'produto', 'vendedor', 'marca', 'cidade', 'estado', 'cep', 'pagamento']:
    df[coluna] = df[coluna].astype(str).apply(corrigir_encoding)


## 🔍 Limpeza de Dados - Tratamento de Data e Hora
O objetivo é garantir que a coluna `data` esteja no formato `YYYY-MM-DD` e a coluna `hora` no formato `HH:MM:SS`.

### Passos:
1. Converter o campo `data` para o formato `YYYY-MM-DD`.
2. Ajustar o campo `hora` para o formato `HH:MM:SS`.


In [33]:
# Limpeza do campo 'data' para o formato 'YYYY-MM-DD'
df['data'] = pd.to_datetime(df['data'], errors='coerce').dt.strftime('%Y-%m-%d')

# Limpeza do campo 'hora' para o formato 'HH:MM:SS'
# Supondo que o formato seja 'HH:MM' ou 'HH:MM:SS'
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')

# Verificar as primeiras linhas após a limpeza
df[['data', 'hora']].head()


,data,hora
0,2021-03-20,23:35:51
1,2020-10-30,09:00:53
2,2021-06-09,15:30:28
3,2022-06-04,08:41:23
4,2019-05-04,13:38:45


# 🔍 Verificar valores inválidos (NaT) nos campos 'data' e 'hora'


In [34]:
# Reconvertendo os campos para datetime para validação
data_invalidas = pd.to_datetime(df['data'], errors='coerce', format='%Y-%m-%d').isna()
hora_invalidas = pd.to_datetime(df['hora'], errors='coerce', format='%H:%M:%S').isna()

# Total de valores inválidos
print("📅 Datas inválidas:", data_invalidas.sum())
print("⏰ Horas inválidas:", hora_invalidas.sum())

# (Opcional) Exibir algumas linhas com problemas
df[data_invalidas | hora_invalidas][['data', 'hora']].head(10)


📅 Datas inválidas: 0
⏰ Horas inválidas: 0


,data,hora


## 👤 Verificação da coluna `cliente`

Nesta etapa, vamos verificar possíveis problemas com os dados de clientes, como:
- Valores nulos ou vazios
- Espaços desnecessários
- Inconsistências de formatação (maiúsculas/minúsculas)
- Registros inválidos ou muito curtos


In [35]:
# Remover espaços desnecessários
df['cliente'] = df['cliente'].astype(str).str.strip()

# Verificar valores nulos ou vazios
clientes_nulos = df['cliente'].isna() | (df['cliente'] == '')
print("👤 Clientes vazios ou nulos:", clientes_nulos.sum())

# Verificar nomes curtos (com 2 ou menos caracteres)
clientes_curto = df['cliente'].str.len() <= 2
print("👤 Clientes com nomes muito curtos (<= 2 caracteres):", clientes_curto.sum())

# Exibir amostra de possíveis problemas
df[clientes_nulos | clientes_curto]['cliente'].value_counts().head(10)


👤 Clientes vazios ou nulos: 0
👤 Clientes com nomes muito curtos (<= 2 caracteres): 0


Series([], Name: count, dtype: int64)

## 🔍 Verificação adicional e normalização de nomes de clientes

Agora, vamos verificar possíveis problemas adicionais, como espaços extras, inconsistências na capitalização e duplicatas. Em seguida, vamos normalizar todos os nomes para **letras maiúsculas**.


In [36]:
# Remover espaços extras (início e fim)
df['cliente'] = df['cliente'].str.strip()

# Normalizar para maiúsculas
df['cliente'] = df['cliente'].str.upper()

# Verificar se há nomes com números ou símbolos (considerando nomes válidos)
# Vamos filtrar apenas se contiver números ou caracteres não alfabéticos
clientes_invalidos = df['cliente'].str.contains(r'[^A-Z\sÇÁÉÍÓÚãâãáà]', regex=True)

print("🚨 Clientes com caracteres inválidos:", clientes_invalidos.sum())

# Exibir amostra de clientes inválidos
df[clientes_invalidos]['cliente'].drop_duplicates().head(10)

# Verificar duplicatas
duplicatas = df['cliente'].duplicated().sum()
print(f"🔁 Total de clientes duplicados: {duplicatas}")


🚨 Clientes com caracteres inválidos: 53132
🔁 Total de clientes duplicados: 368104


In [37]:
# Verificar se há registros idênticos (mesmo cliente, mesmo produto, mesma data e hora)
duplicatas_identicas = df.duplicated(subset=['cliente', 'produto', 'data', 'hora'], keep=False)
print(f"🚨 Total de registros idênticos (mesmo cliente, produto e data/hora): {duplicatas_identicas.sum()}")

# Exibir registros idênticos
df[duplicatas_identicas].head(20)


🚨 Total de registros idênticos (mesmo cliente, produto e data/hora): 136359


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS ARAUJO KUHN,Queijo Mussarela,"R$ 16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"R$ 3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,"R$ 9,48",2,18.96,Em SeparaÃ§Ã£o,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,Café,"R$ 10,16",1,20.16,Entregue com Sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,Sabonete,"R$ 1,25",2,13.00,Em SeparaÃ§Ã£o,São João del-Rei,MG,Brasil,36300-000,10.5,Boleto,GABRIEL QUEIROZ DE AGUIAR,Dove
10,43813,2021-06-04,21:07:04,PEDRO MATOS LIMA DA CUNHA,Açúcar,"R$ 3,59",1,3.59,Aguardando Pagamento,São Gonçalo,RJ,Brasil,24400-000,0.0,Transferência Bancária,SAMUEL HENRIQUE CAÇADOR,Caravelas
12,10784,2020-07-03,22:34:21,MATEUS VICTOR ALVES,Manteiga,"R$ 6,61",8,71.88,Em Transporte,Duque de Caxias,RJ,Brasil,25000-000,19.0,Pix à Vista,SAMUEL HENRIQUE CAÇADOR,Itambé
13,1271,2021-04-18,10:56:01,CARLOS ASSIS NOGUEIRA SILVA,Papel Higiênico,"R$ 3,89",12,46.68,Em SeparaÃ§Ã£o,Macaé,RJ,Brasil,27900-000,0.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Neve
15,5650,2021-06-11,14:18:25,VICTOR DA CUNHA TEIXEIRA,Café,"R$ 5,29",1,5.29,Em SeparaÃ§Ã£o,Viçosa,MG,Brasil,36570-000,0.0,Transferência Bancária,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,3 Corações
17,24847,2022-06-23,12:15:47,THIAGO LACERDA PORTUGAL DA SILVA,Açúcar,"R$ 3,75",3,24.25,Em SeparaÃ§Ã£o,Juiz de Fora,MG,Brasil,36000-000,13.0,Transferência Bancária,SAMUEL HENRIQUE CAÇADOR,Caravelas


In [38]:
# Convertendo todos os nomes para maiúsculas
df['cliente'] = df['cliente'].str.upper()


### 🔢 Verificação de Formato de Valores Monetários

Este código verifica se os valores na coluna `valor` estão no formato correto de moeda brasileira: "R$ 10,23".

**Como funciona:**

1. **Expressão Regular**: Utiliza uma regex para garantir que o valor esteja no padrão correto.
2. **Filtragem de Valores Inválidos**: Identifica os valores que não seguem o formato esperado.
3. **Exibição dos Valores Inválidos**: Mostra os primeiros valores fora do padrão.

O objetivo é identificar e corrigir valores que não estão no formato correto.


In [39]:
import re

# Remover espaços em branco extras
df['valor'] = df['valor'].str.strip()

# Criar uma expressão regular para verificar o formato esperado "R$ xx,xx"
pattern = r"^R\$\s\d{1,3}(\.\d{3})*(,\d{2})?$"

# Verificar quais valores não estão no formato esperado
invalid_values = df[~df['valor'].str.match(pattern, na=False)]

# Mostrar os valores inválidos
invalid_values[['valor']].head()


,valor
64,"R$ 586,8800000000001"
72,"R$ 14,1"
73,"R$ 2,9"
82,"R$ 2,9"
92,"R$ 14,6"


In [40]:
import pandas as pd
import numpy as np

# Converter a coluna 'valor' para numérico após limpar o formato
df['valor'] = pd.to_numeric(
    df['valor'].astype(str)
        .str.replace(r'R\$', '', regex=True)
        .str.replace(',', '.'),
    errors='coerce'
)

# Verificar e reportar os valores não numéricos
valores_nao_numericos = df['valor'].isna().sum()
if valores_nao_numericos > 0:
    print(f"🔍 Total de valores não numéricos ou não convertidos: {valores_nao_numericos}")
else:
    print("✔️ Todos os valores foram convertidos com sucesso para numérico.")

# Cálculo dos quartis e IQR
Q1 = df['valor'].quantile(0.25)
Q3 = df['valor'].quantile(0.75)
IQR = Q3 - Q1

# Limites para detecção de outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Identificar outliers
outliers = df[(df['valor'] < limite_inferior) | (df['valor'] > limite_superior)]
if not outliers.empty:
    print("🚨 Outliers encontrados:")
    print(outliers[['valor']].head())
else:
    print("✔️ Nenhum outlier encontrado.")

# Substituir outliers por NaN
df.loc[(df['valor'] < limite_inferior) | (df['valor'] > limite_superior), 'valor'] = np.nan

# Preencher com a mediana
mediana = df['valor'].median()
df['valor'] = df['valor'].fillna(mediana)

# Exibir resultado final
print("\n✔️ Dados tratados (outliers substituídos pela mediana):")
print(df.head())




🔍 Total de valores não numéricos ou não convertidos: 83
🚨 Outliers encontrados:
     valor
23   30.35
35   33.33
62   23.89
64  586.88
99  454.57

✔️ Dados tratados (outliers substituídos pela mediana):
   id_da_compra        data      hora                      cliente  \
0         13679  2021-03-20  23:35:51            LUCAS ARAUJO KUHN   
1         28070  2020-10-30  09:00:53         MICAEL SOUZA RONCETE   
2         47484  2021-06-09  15:30:28    FELIPE AUGUSTO NERY SILVA   
3         20809  2022-06-04  08:41:23          LEVI RIBEIRO AMORIM   
4         47123  2019-05-04  13:38:45  GABRIEL MATOS LIMA DA CUNHA   

            produto  valor  quantidade   total                status  \
0  Queijo Mussarela  16.87          13  239.31  Pagamento Confirmado   
1   Molho de Tomate   3.25           3    9.75  Pagamento Confirmado   
2      Água Mineral   1.63           9   36.67  Pagamento Confirmado   
3            Carvão   8.74           4   54.96        Em SeparaÃ§Ã£o   
4              C

### 🧹 Tratamento da Coluna `valor` e Continuidade do Pré-processamento

Anteriormente, realizamos um pré-processamento na coluna `valor` para garantir a qualidade e consistência dos dados:

- **Conversão de dados**: Transformamos a coluna `valor` de texto para o tipo numérico (`float`), removendo o símbolo `R$` e substituindo vírgulas por pontos para padronizar o formato monetário.
- **Detecção e tratamento de outliers**: Identificamos valores atípicos (outliers) usando o método do IQR (Intervalo Interquartílico) e os substituímos pela mediana da coluna para evitar distorções na análise.
- **Tratamento de valores ausentes**: Valores não convertidos foram tratados como `NaN` e preenchidos também com a mediana.

---

Agora, vamos **aplicar o mesmo processo às colunas `quantidade`, `total` e `frete`**, pois essas colunas:

- Também devem estar no formato numérico para permitir análises estatísticas e cálculos corretos.
- Podem conter outliers que afetam a qualidade dos dados.
- Precisam estar consistentes para garantir que o campo `total` obedeça à fórmula esperada:

```
    total = valor * quantidade + frete
```

Além disso, vamos verificar se **existem valores ausentes** nessas colunas e definir uma estratégia de tratamento adequada (remoção ou preenchimento por mediana).



## 📋 Conversão das colunas para float e arredondamento
Convertendo as colunas de valor, quantidade, frete e total para float, arredondando com round(2) para padronização.

In [41]:
# Removendo símbolos e transformando strings numéricas
def limpar_valor(col):
    return pd.to_numeric(
        col.astype(str)
           .str.replace(r'R\$', '', regex=True)
           .str.replace(',', '.'),
        errors='coerce'
    )

# Aplicando para as colunas relevantes
df['valor'] = limpar_valor(df['valor']).round(2)
df['frete'] = limpar_valor(df['frete']).round(2)
df['quantidade'] = limpar_valor(df['quantidade']).round(2)
df['total'] = limpar_valor(df['total']).round(2)

## 🧪 Verificando valores faltantes após a conversão

In [42]:
print("Valores nulos por coluna após conversão:")
print(df[['valor', 'frete', 'quantidade', 'total']].isna().sum())


Valores nulos por coluna após conversão:
valor            0
frete         7371
quantidade       0
total         3685
dtype: int64


## 🧹 Tratamento de outliers e negativos no frete
Identificando e corrigindo outliers, valores negativos e NaN na coluna frete.

In [43]:
# Detectando outliers pelo método IQR
Q1 = df['frete'].quantile(0.25)
Q3 = df['frete'].quantile(0.75)
IQR = Q3 - Q1

limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Criando máscara de valores problemáticos
frete_problema = (
    (df['frete'] < 0) |
    (df['frete'].isna()) |
    (df['frete'] < limite_inferior) |
    (df['frete'] > limite_superior)
)

print(f"Entradas com frete inválido: {frete_problema.sum()}")


Entradas com frete inválido: 14533


## 🏙️ Agrupando frete mais comum por cidade



In [44]:
# Obter o frete mais comum por cidade (moda)
frete_mais_comum = df[~frete_problema].groupby('cidade')['frete'].agg(lambda x: x.mode().iloc[0])
print("\nFretes mais comuns por cidade:")
print(frete_mais_comum)



Fretes mais comuns por cidade:
cidade
Angra dos Reis            0.0
Astolfo Dutra             0.0
Barbacena                12.0
Barroso                   0.0
Belford Roxo             19.0
Belo Horizonte            0.0
Bicas                     0.0
Cabo Frio                22.0
Campinas                  0.0
Campos dos Goytacazes     0.0
Carandaí                  9.0
Cataguases                0.0
Congonhas                10.5
Conselheiro Lafaiete      0.0
Coronel Xavier Chaves     0.0
Duque de Caxias           0.0
Ewbank da Câmara          0.0
Guarulhos                 0.0
Itaboraí                  0.0
Juiz de Fora             13.0
Lagoa Dourada            10.0
Leopoldina                0.0
Lima Duarte               0.0
Macaé                     0.0
Magé                     18.0
Mariana                   0.0
Matias Barbosa           10.0
Muriaé                    0.0
Niterói                   0.0
Nova Iguaçu               0.0
Ouro Branco               0.0
Ouro Preto               12.5
P

## 🔁 Corrigindo fretes inválidos com base na cidade

In [45]:
# Preencher fretes inválidos com a moda por cidade
def substituir_frete(row):
    if frete_problema.loc[row.name]:
        return frete_mais_comum.get(row['cidade'], np.nan)
    return row['frete']

df['frete'] = df.apply(substituir_frete, axis=1).round(2)

print("\n✔️ Fretes corrigidos com base na cidade.")



✔️ Fretes corrigidos com base na cidade.


## 📐 Verificação da fórmula do total: `total = valor * quantidade + frete`


In [46]:
# Calculando o total teórico
df['total_calculado'] = (df['valor'] * df['quantidade'] + df['frete']).round(2)

# Verificando divergências
inconsistentes = df[df['total'] != df['total_calculado']]
print(f"\n⚠️ Linhas com total inconsistente: {inconsistentes.shape[0]}")
print(inconsistentes[['valor', 'quantidade', 'frete', 'total', 'total_calculado']].head())



⚠️ Linhas com total inconsistente: 31922
    valor  quantidade  frete   total  total_calculado
23   4.01           3    8.5   99.55            20.53
35   4.01           6   20.0  219.98            44.06
62   4.01           7    0.0  167.23            28.07
64   4.01           6    0.0   26.88            24.06
73   2.90          13    0.0   48.70            37.70


## 🛠️ Corrigindo valores de `total` com base na fórmula


In [47]:
df['total'] = df['total_calculado']
df.drop(columns=['total_calculado'], inplace=True)
print("\n✔️ Coluna 'total' corrigida com base na fórmula.")



✔️ Coluna 'total' corrigida com base na fórmula.


### Limpeza da Coluna `status`

Agora, vamos focar na limpeza da coluna `status`. Essa coluna pode conter valores inconsistentes, como espaços extras, variações de maiúsculas/minúsculas e valores inesperados que precisam ser tratados. O objetivo é garantir que os dados dessa coluna sejam uniformes e estejam no formato correto para análise e processamento.

#### Passos a serem seguidos:

1. **Verificar valores únicos**: Vamos listar todos os valores distintos presentes na coluna `status` para identificar inconsistências.
2. **Contagem de valores**: A contagem de frequências de cada valor ajudará a ver se algum valor aparece de maneira inesperada ou errada.
3. **Limpeza dos valores**:
   - Removeremos espaços extras que possam estar presentes em torno


In [48]:
# Verificar todos os valores únicos na coluna 'status'
valores_unicos_status = df['status'].unique()

# Exibir os valores únicos
print(valores_unicos_status)


['Pagamento Confirmado' 'Em SeparaÃ§Ã£o' 'Entregue com Sucesso'
 'Aguardando Pagamento' 'Em Transporte' 'AP' 'Entregue' 'PC' 'Sep'
 'Separando' 'Pgto Confirmado' 'Transp' 'Transportando' 'Entg'
 'aguardando pagamento' 'Aguardando Pgto']


### 🔍 Limpeza e Padronização da Coluna `status`

Agora, vamos proceder com a limpeza e padronização dos valores da coluna `status`. O objetivo é garantir que todos os valores estejam consistentes, facilitando a análise dos dados. A coluna contém várias variações de status, incluindo abreviações, erros de codificação e diferenças de maiúsculas/minúsculas.

#### Ações que serão realizadas:

1. **Padronização de abreviações**: Algumas abreviações, como "AP", "PC", "Sep", "Transp" e "Entg", são formas curtas de status conhecidos. Vamos substituí-las pelas versões completas e consistentes, como "Pagamento Confirmado", "Separando", "Transportando", etc.

2. **Correção de erros de codificação**: O valor "Em SeparaÃ§Ã£o" contém caracteres corrompidos, provavelmente devido a um problema de codificação de caracteres. Vamos corrigir para "Em Separação".

3. **Uniformização de maiúsculas e minúsculas**: Para evitar discrepâncias como "Aguardando Pagamento" e "aguardando pagamento", vamos normalizar todos os valores para minúsculas.

4. **Tratamento de valores ausentes**: Caso existam valores nulos ou faltantes, vamos definir a melhor estratégia, seja removendo os registros ou imputando valores apropriados.

Após essas modificações, a coluna estará mais limpa e padronizada, pronta para análises e relatórios.


In [49]:
# Dicionário de mapeamento de abreviações para valores completos

status_map = {
    'Em SeparaÃ§Ã£o' : 'em separação',
    'AP' : 'aguardando pagamento',
    'Entregue com Sucesso' : 'entregue',
    'Sep' : 'em separação',
    'PC' : 'pagamento confirmado',
    'Separando' : 'em separação',
    'Pgto Confirmado' : 'pagamento confirmado',
    'Transp' : 'em transporte',
    'Transportando': 'em transporte',
    'Entg' : 'entregue',
    'Aguardando Pgto' : 'aguardando pagamento',
}


# Substituir as abreviações e corrigir a codificação
df['status'] = df['status'].replace(status_map)

# Corrigir possíveis problemas com espaços extras
df['status'] = df['status'].str.replace(r'\s+', ' ', regex=True)  # Substitui múltiplos espaços por um único

# Remover espaços extras antes ou depois dos textos
df['status'] = df['status'].str.strip()

# Uniformizar os valores para minúsculas
df['status'] = df['status'].str.lower()

# Verificar se há valores ausentes (nulos) e tratá-los
df['status'] = df['status'].fillna('não definido')

# Verificar o resultado
df['status'].value_counts()


status
em separação            110609
pagamento confirmado    109870
aguardando pagamento     74938
em transporte            36840
entregue                 36495
Name: count, dtype: int64

### Tratamento da Coluna `pagamento`

Agora, vamos realizar a limpeza e padronização da coluna `pagamento` para garantir que os valores estejam consistentes e sem erros. O processo será o seguinte:

1. **Verificar as Ocorrências Atuais**:
   - Vamos usar `value_counts()` para listar todas as ocorrências na coluna `pagamento` e verificar se existem abreviações, erros de digitação ou valores inconsistentes. Isso nos ajudará a identificar se há valores como "boleto", "pix", "cartão", entre outros, de forma inconsistente ou com variações que precisam ser unificadas.

2. **Aplicar Mapeamento de Valores**:
   - Iremos mapear os diferentes tipos de pagamento, como "boleto", "pix", "cartão", etc., para valores consistentes. Isso inclui corrigir abreviações, erros de digitação ou variações de capitalização. Por exemplo, se houver "boleto bancário" e "boleto", ambos devem ser mapeados para o mesmo valor.

3. **Limpeza de Espaços Extras**:
   - Vamos remover espaços extras antes e depois dos valores, e também substituir múltiplos espaços consecutivos por um único espaço, caso haja algum.

4. **Transformar para Minúsculas**:
   - Para uniformizar a coluna, vamos garantir que todos os valores estejam em minúsculas, eliminando discrepâncias relacionadas à capitalização, como "BOLETO" e "boleto", que devem ser tratados da mesma forma.

5. **Tratar Valores Ausentes**:
   - Se houver valores ausentes (nulos) na coluna `pagamento`, vamos tratá-los substituindo-os por um valor padrão, como 'Não Definido' ou 'Não Informado', para evitar dados faltantes.

6. **Rever os Resultados**:
   - Após todas as modificações, vamos verificar as ocorrências novamente para garantir que todos os valores estejam consistentes e que a coluna esteja pronta para análise.

Esse processo garantirá que a coluna `pagamento` tenha dados mais limpos, consistentes e prontos para serem usados em análises e relatórios, além de facilitar a categorização por tipo de pagamento.


In [50]:
# Verificar as ocorrências de cada tipo na coluna 'pagamento'
df['pagamento'].value_counts()


pagamento
Cartão de Crédito         108459
Transferência Bancária    106721
Pix                       106504
Boleto                     36005
Pix à Vista                 1717
Cartão Crédito à Vista      1677
Pagamento Instantâneo       1676
Cartão Crédito              1647
TED                         1633
DOC                         1602
Boleto à Vista               593
Boleto no Dinheiro           518
Name: count, dtype: int64

In [51]:
# Dicionário de mapeamento de abreviações para valores completos no campo 'pagamento'
pagamento_map = {
    'Boleto à Vista': 'boleto',
    'Boleto no Dinheiro': 'boleto',
    'Cartão Crédito à Vista': 'crédito à vista',
    'Cartão de Crédito': 'cartão de crédito',
    'TED': 'transferência bancária',
    'DOC': 'transferência bancária',
    'Pagamento Instantâneo': 'pix',
    'Pix à Vista': 'pix',
}

# Substituir valores incorretos
df['pagamento'] = df['pagamento'].replace(pagamento_map)

# Corrigir possíveis problemas com espaços extras
df['pagamento'] = df['pagamento'].str.replace(r'\s+', ' ', regex=True)  # Substitui múltiplos espaços por um único

# Remover espaços extras antes ou depois dos textos
df['pagamento'] = df['pagamento'].str.strip()

# Uniformizar os valores para minúsculas
df['pagamento'] = df['pagamento'].str.lower()

# Verificar se há valores ausentes (nulos) e tratá-los
df['pagamento'] = df['pagamento'].fillna('não definido')

# Verificar o resultado
df['pagamento'].value_counts()


pagamento
transferência bancária    109956
pix                       109897
cartão de crédito         108459
boleto                     37116
crédito à vista             1677
cartão crédito              1647
Name: count, dtype: int64

In [52]:
# Verificar valores únicos na coluna de pagamento
valores_unicos_pagamento = df['pagamento'].unique()
print("Valores únicos em 'pagamento':")
print(valores_unicos_pagamento)

# Verificar se há valores ausentes ou nulos
valores_nulos_pagamento = df['pagamento'].isnull().sum()
print(f"Valores nulos na coluna 'pagamento': {valores_nulos_pagamento}")

Valores únicos em 'pagamento':
['cartão de crédito' 'pix' 'transferência bancária' 'boleto'
 'crédito à vista' 'cartão crédito']
Valores nulos na coluna 'pagamento': 0


### 🌍 Limpeza e Padronização da Coluna `estado`

Agora iremos realizar a limpeza e padronização da coluna `estado` do nosso conjunto de dados. O objetivo é garantir consistência nos valores representando os estados brasileiros.

#### Etapas previstas:
1. **Inspeção dos valores únicos**: Verificar quais valores diferentes estão presentes na coluna `estado`, identificando possíveis erros de digitação, abreviações inconsistentes ou valores inválidos.
2. **Criação de um dicionário de mapeamento**: Construir um dicionário para padronizar os nomes e siglas dos estados. Por exemplo, transformar "sp", "SP ", "São Paulo", etc., em "SP".
3. **Aplicação do mapeamento**: Substituir os valores inconsistentes pelos padronizados, utilizando o dicionário de mapeamento.
4. **Tratamento de valores ausentes**: Se houver estados ausentes (`NaN`),


In [53]:
# Verificar valores únicos na coluna de estado
valores_unicos_estado = df['estado'].unique()
print("Valores únicos em 'estado':")
print(valores_unicos_estado)

# Verificar se há valores ausentes ou nulos
valores_nulos_estado = df['estado'].isnull().sum()
print(f"Valores nulos na coluna 'estado': {valores_nulos_estado}")


Valores únicos em 'estado':
['RJ' 'MG' 'SP' 'MTSa' 'São Paulo' 'RS' 'PR' 'PSC']
Valores nulos na coluna 'estado': 0


In [54]:
# Dicionário de mapeamento para padronizar os estados
estado_map = {
    'São Paulo': 'SP',
    'MTSa': 'MT',
    'PSC': 'SC'
}

# Aplicar o mapeamento
df['estado'] = df['estado'].replace(estado_map)

# Verificar os valores únicos após a substituição
df['estado'].unique()


array(['RJ', 'MG', 'SP', 'MT', 'RS', 'PR', 'SC'], dtype=object)

## 🏙️ Conexão entre as Colunas 'CEP' e 'Cidade'

### Objetivo:
O objetivo de realizar uma validação combinada entre as colunas **CEP** e **Cidade** é garantir que os dados de localização estejam consistentes, completos e correspondam corretamente entre si. Isso é crucial para garantir a qualidade e precisão dos dados, além de permitir análises e operações futuras sem erros ou inconsistências.

### Por que Validar 'CEP' e 'Cidade' Juntas?

1. **Consistência de Dados**:
   - O **CEP** é um código postal utilizado para identificar a localização geográfica de um endereço, e ele é diretamente relacionado à **cidade** em que está localizado. Portanto, é importante que, ao validarmos o **CEP**, também verifiquemos se ele corresponde à cidade associada.
   - Se a cidade for válida, mas o **CEP** não corresponder ou estiver em um formato inválido, isso pode indicar um erro nos dados ou uma incoerência no registro.

2. **Garantir Integridade Geográfica**:
   - O processo de validação cruzada entre o **CEP** e a **Cidade** permite evitar casos em que o **CEP** esteja registrado para uma cidade inexistente ou errada.
   - Também ajuda a filtrar entradas onde o **CEP** pode ser incompleto ou mal formatado, assegurando que o formato do **CEP** esteja correto, e que ele pertença à cidade informada.

3. **Prevenção de Dados Inconsistentes**:
   - Ao realizar a validação combinada, evitamos problemas como:
     - Cidades com **CEPs** inválidos.
     - Cidades não existentes ou mal escritas associadas a **CEPs** reais.
     - Dados faltantes, que podem ser preenchidos corretamente se a verificação for feita com base em ambas as colunas.

### Como Funciona a Validação Cruzada:

1. **Validação da Cidade**:
   - A primeira etapa consiste em garantir que o nome da cidade na coluna **cidade** seja válido. Isso envolve verificar se a cidade contém apenas caracteres alfabéticos e espaços (evitar números, caracteres especiais e valores nulos).
   - Caso algum valor da cidade seja inválido (como cidades com números ou caracteres não alfabéticos), ele será convertido para `NaN`.

2. **Validação do CEP**:
   - O **CEP** será verificado para garantir que esteja no formato correto (ex: `00000-000`).
   - O **CEP** será validado para garantir que ele não contenha valores incompletos, nulos ou inválidos.
   - Em paralelo, é importante que o **CEP** se associe corretamente à **cidade**, caso haja necessidade de validação geográfica adicional.

3. **Validação Combinada**:
   - Após a verificação individual, será realizada uma análise de consistência cruzada entre o **CEP** e a **Cidade** para garantir que a cidade associada ao **CEP** seja válida.
   - Caso o **CEP** ou a **Cidade** apresentem discrepâncias (como uma cidade que não corresponda a um **CEP** válido ou vice-versa), essas entradas serão marcadas como `NaN` para posterior análise e correção.



In [55]:
# Verificar valores únicos e nulos nas colunas de 'cidade' e 'cep'
valores_unicos_cidade = df['cidade'].unique()
valores_nulos_cidade = df['cidade'].isnull().sum()

valores_unicos_cep = df['cep'].unique()
valores_nulos_cep = df['cep'].isnull().sum()

# Exibir as informações
print("Valores únicos em 'cidade':")
print(valores_unicos_cidade)
print(f"Valores nulos na coluna 'cidade': {valores_nulos_cidade}\n")

print("Valores únicos em 'cep':")
print(valores_unicos_cep)
print(f"Valores nulos na coluna 'cep': {valores_nulos_cep}\n")


Valores únicos em 'cidade':
['Niterói' 'Mariana' 'Cabo Frio' 'Campos dos Goytacazes'
 'Conselheiro Lafaiete' 'Resende Costa' 'São Paulo' 'Bicas'
 'São João del-Rei' 'São Gonçalo' 'São Bernardo do Campo'
 'Duque de Caxias' 'Macaé' 'Viçosa' 'Juiz de Fora' 'Guarulhos'
 'Matias Barbosa' 'Campinas' 'Barroso' 'Carandaí' 'Barbacena' 'Itaboraí'
 'Muriaé' 'Petrópolis' 'Ouro Branco' 'Prados' 'Santos Dumont' 'Cataguases'
 'São João de Meriti' 'Ubá' 'Palma' 'Simão Pereira' 'Magé'
 'Ewbank da Câmara' 'Santana do Garambéu' 'Astolfo Dutra'
 'São João Nepomuceno' 'Belo Horizonte' 'Tocantins' 'Congonhas'
 'São Vicente de Minas' 'Coronel Xavier Chaves' 'Leopoldina'
 'Belford Roxo' 'Ritápolis' 'Lagoa Dourada' 'Rio de Janeiro' 'Nova Iguaçu'
 'São Tiago' 'São José dos Campos' 'Volta Redonda' 'Lima Duarte'
 'Santo André' 'Tiradentes' 'Ouro Preto' 'Angra dos Reis']
Valores nulos na coluna 'cidade': 0

Valores únicos em 'cep':
['24000-000' '35420-000' '28900-000' '28000-000' '36400-000' '36340-000'
 '01000-00

In [56]:
# Exemplo de dicionário com CEPs válidos para algumas cidades (o mapeamento deve ser mais completo)
cidade_para_cep = {
    'Niterói': '24000-000',
    'Mariana': '35420-000',
    'Cabo Frio': '28900-000',
    'Campos dos Goytacazes': '28000-000',
    'Conselheiro Lafaiete': '36400-970',
    'Resende Costa': '36340-000',
    'São Paulo': '01000-000',
    'Bicas': '36600-032',
    'São João del-Rei': '36300-001',
    'São Gonçalo': '24410-000',
    'São Bernardo do Campo': '09600-004',
    'Duque de Caxias': '25000-000',
    'Macaé': '27975-170',
    'Viçosa': '36570-000',
    'Juiz de Fora': '36000-000',
    'Guarulhos': '07000-000',
    'Matias Barbosa': '36120-000',
    'Campinas': '13000-000',
    'Barroso': '36212-000',
    'Carandaí': '36280-000',
    'Barbacena': '36200-000',
    'Itaboraí': '24800-000',
    'Muriaé': '36880-000',
    'Petrópolis': '25600-000',
    'Ouro Branco': '36420-000',
    'Prados': '36320-000',
    'Santos Dumont': '36240-000',
    'Cataguases': '36770-000',
    'São João de Meriti': '25500-000',
    'Ubá': '36500-000',
    'Palma': '36710-000',
    'Simão Pereira': '36123-000',
    'Magé': '25900-000',
    'Ewbank da Câmara': '36108-000',
    'Santana do Garambéu': '36146-000',
    'Astolfo Dutra': '36780-000',
    'São João Nepomuceno': '36196-000',
    'Belo Horizonte': '30100-000',
    'Tocantins': '36680-000',
    'Congonhas': '36415-000',
    'São Vicente de Minas': '36510-000',
    'Coronel Xavier Chaves': '36415-000',
    'Leopoldina': '36770-000',
    'Belford Roxo': '26100-000',
    'Ritápolis': '36335-000',
    'Lagoa Dourada': '36240-000',
    'Rio de Janeiro': '20000-000',
    'Nova Iguaçu': '26200-000',
    'São Tiago': '36280-000',
    'São José dos Campos': '12200-000',
    'Volta Redonda': '27200-000',
    'Lima Duarte': '36140-000',
    'Santo André': '09000-000',
    'Tiradentes': '36325-000',
    'Ouro Preto': '35400-000',
    'Angra dos Reis': '23900-000'
}

# Função para corrigir CEPs inválidos
def corrigir_cep(row):
    if 'X' in row['cep'] or len(row['cep']) != 9:
        return cidade_para_cep.get(row['cidade'], row['cep'])  # Substitui se a cidade tiver um CEP válido mapeado
    return row['cep']

# Aplicar a função ao DataFrame
df['cep'] = df.apply(corrigir_cep, axis=1)

# Salvar o DataFrame atualizado
df.to_csv('vendas_limpo.csv', index=False)


In [57]:
# Verificar valores únicos e nulos nas colunas de 'cidade' e 'cep'
valores_unicos_cidade = df['cidade'].unique()
valores_nulos_cidade = df['cidade'].isnull().sum()

valores_unicos_cep = df['cep'].unique()
valores_nulos_cep = df['cep'].isnull().sum()

# Exibir as informações
print("Valores únicos em 'cidade':")
print(valores_unicos_cidade)
print(f"Valores nulos na coluna 'cidade': {valores_nulos_cidade}\n")

print("Valores únicos em 'cep':")
print(valores_unicos_cep)
print(f"Valores nulos na coluna 'cep': {valores_nulos_cep}\n")


Valores únicos em 'cidade':
['Niterói' 'Mariana' 'Cabo Frio' 'Campos dos Goytacazes'
 'Conselheiro Lafaiete' 'Resende Costa' 'São Paulo' 'Bicas'
 'São João del-Rei' 'São Gonçalo' 'São Bernardo do Campo'
 'Duque de Caxias' 'Macaé' 'Viçosa' 'Juiz de Fora' 'Guarulhos'
 'Matias Barbosa' 'Campinas' 'Barroso' 'Carandaí' 'Barbacena' 'Itaboraí'
 'Muriaé' 'Petrópolis' 'Ouro Branco' 'Prados' 'Santos Dumont' 'Cataguases'
 'São João de Meriti' 'Ubá' 'Palma' 'Simão Pereira' 'Magé'
 'Ewbank da Câmara' 'Santana do Garambéu' 'Astolfo Dutra'
 'São João Nepomuceno' 'Belo Horizonte' 'Tocantins' 'Congonhas'
 'São Vicente de Minas' 'Coronel Xavier Chaves' 'Leopoldina'
 'Belford Roxo' 'Ritápolis' 'Lagoa Dourada' 'Rio de Janeiro' 'Nova Iguaçu'
 'São Tiago' 'São José dos Campos' 'Volta Redonda' 'Lima Duarte'
 'Santo André' 'Tiradentes' 'Ouro Preto' 'Angra dos Reis']
Valores nulos na coluna 'cidade': 0

Valores únicos em 'cep':
['24000-000' '35420-000' '28900-000' '28000-000' '36400-000' '36340-000'
 '01000-00

# 📝 Análise de Produtos e Marcas - Documentação de Processos

## 🔍 Objetivo

O objetivo dessa etapa foi realizar uma análise exploratória na base de dados, com foco em verificar e corrigir valores de produtos e marcas. Também foram realizadas verificações para identificar dados faltantes e inconsistências nos nomes dos produtos. Para isso, foi utilizado um dicionário de correções para garantir que os nomes dos produtos estejam uniformizados.

## 🛠️ Etapas Realizadas

### 1. **Verificação de Valores Nulos**
Antes de iniciar qualquer análise, é importante verificar se existem valores nulos na base de dados. Isso ajuda a identificar possíveis problemas de qualidade de dados. Foram realizadas as seguintes verificações:

- **Produtos Nulos**: Verificamos quantos produtos estão ausentes (`NaN`).
- **Marcas Nulas**: Verificamos quantas marcas estão ausentes (`NaN`).

```python
# Verificar valores nulos
print("🔍 Produtos nulos:", df['produto'].isna().sum())
print("🔍 Marcas nulas:", df['marca'].isna().sum())


In [58]:
# Verificar valores nulos
print("🔍 Produtos nulos:", df['produto'].isna().sum())
print("🔍 Marcas nulas:", df['marca'].isna().sum())

# Verificar valores únicos
print("\n📦 Produtos únicos:")
print(df['produto'].value_counts())

print("\n🏷️ Marcas únicas:")
print(df['marca'].value_counts())

# Checar por valores estranhos (ex: espaços extras, letras maiúsculas/minúsculas diferentes)
print("\n🔎 Valores de 'produto' únicos (ordenados):")
print(sorted(df['produto'].dropna().unique()))

print("\n🔎 Valores de 'marca' únicos (ordenados):")
print(sorted(df['marca'].dropna().unique()))


🔍 Produtos nulos: 0
🔍 Marcas nulas: 0

📦 Produtos únicos:
produto
Pasta de Dente      25710
Queijo Mussarela    25634
Sabonete            24278
Manteiga            23769
Café                21916
                    ...  
Macirrão                1
Deqergente              1
Cafc                    1
Queijo Mussarelz        1
Deterwente              1
Name: count, Length: 99, dtype: int64

🏷️ Marcas únicas:
marca
Dove            15224
Itambé           9553
Colgate          8819
Porto Alegre     8806
Quatá            8804
                ...  
Brilhante        1001
Tixan             990
Genérico          553
Marca-Brás        515
Ki-Brasa          515
Name: count, Length: 88, dtype: int64

🔎 Valores de 'produto' únicos (ordenados):
['Amaciante', 'Amaciante#$@!', 'Amaciayte', 'Arroc', 'Arroz', 'Arroz#$@!', 'Açúcar', 'Açúcar#$@!', 'Açúcaz', 'Biscoito Recheado', 'Biscoito Recheado#$@!', 'Biscoitq Recheado', 'Cafc', 'Caff', 'Caft', 'Café', 'Café#$@!', 'Carvão', 'Carvão#$@!', 'Cerveja', 'Cerve

## Análise de Produtos por Marca
Queremos entender quantos produtos diferentes existem para cada marca. Para isso, usamos a função groupby para contar o número de produtos únicos por marca.

In [59]:
# Verificar quantos produtos únicos cada marca tem
produtos_por_marca = df.groupby('marca')['produto'].nunique().sort_values(ascending=False)
print("\n📊 Produtos únicos por marca:")
print(produtos_por_marca)


📊 Produtos únicos por marca:
marca
Dove          7
3 Corações    6
Pantene       5
Itambé        5
Seda          5
             ..
Tixan         2
União         2
Urbano        2
Veja          2
Wickbold      2
Name: produto, Length: 88, dtype: int64


# 🧹 Limpeza de Dados com Dicionário de Correções

 Durante a análise, encontramos vários erros de digitação e variações nos nomes dos produtos. Para corrigir isso de forma eficiente, utilizamos um **dicionário de correções**. O dicionário contém pares de valores, onde as chaves representam os nomes incorretos ou variantes, e os valores correspondem aos nomes corretos dos produtos.

### Abaixo, mostramos um exemplo de como os nomes dos produtos foram corrigidos:

In [60]:
dict_produtos_corrigidos = {
    'Cafc': 'Café', 'Caff': 'Café', 'Caft': 'Café', 'Clfé': 'Café', 'Cnfé': 'Café',
    'Café#$@!': 'Café',

    'Qbeijo Mussarela': 'Queijo Mussarela',
    'Queijo Mussarelz': 'Queijo Mussarela',
    'Queijo Mussarela#$@!': 'Queijo Mussarela',

    'zabonete': 'Sabonete', 'Sabonepe': 'Sabonete',
    'Sabonete#$@!': 'Sabonete',

    'Deqergente': 'Detergente', 'Deterwente': 'Detergente',
    'Detergente#$@!': 'Detergente',

    'Desinfekante': 'Desinfetante', 'Desinfetanue': 'Desinfetante',
    'Desinfetante#$@!': 'Desinfetante',

    'Macirrão': 'Macarrão', 'Macawrão': 'Macarrão', 'Majarrão': 'Macarrão', 'Mqcarrão': 'Macarrão',
    'Macarrão#$@!': 'Macarrão',

    'Papel Twalha': 'Papel Toalha', 'Papel qoalha': 'Papel Toalha',
    'Papel Toalha#$@!': 'Papel Toalha',

    'Condibionador': 'Condicionador', 'Condicioiador#$@!': 'Condicionador', 'Condicionador#$@!': 'Condicionador',

    'Água Mineras': 'Água Mineral', 'Água Mineual': 'Água Mineral',
    'Água Mineral#$@!': 'Água Mineral',

    'Arroc': 'Arroz', 'Arroz#$@!': 'Arroz',

    'Açúcaz': 'Açúcar', 'Açúcar#$@!': 'Açúcar',

    'Vinho#$@!': 'Vinho',

    'Shampoo#$@!': 'Shampoo',

    'Amaciante#$@!': 'Amaciante', 'Amaciayte': 'Amaciante',

    'Scl': 'Sal', 'Sal#$@!': 'Sal',

    'Presuntd': 'Presunto', 'Presunto#$@!': 'Presunto',

    'Sabão em Pó#$@!': 'Sabão em Pó',
    'sabão em pó': 'Sabão em Pó',

    'Leite Integral#$@!': 'Leite Integral',

    'ieijão': 'Feijão', 'Feijão#$@!': 'Feijão',

    'Pão de Forma#$@!': 'Pão de Forma',
    'pão de forma': 'Pão de Forma',

    'Sucoyde Laranja': 'Suco de Laranja', 'Suco de Laranja#$@!': 'Suco de Laranja',
    'suco de laranja': 'Suco de Laranja',

    'pasta de dente#$@!': 'Pasta de Dente',
    'pasta de dente': 'Pasta de Dente',

    'manteigt': 'Manteiga',
    'manteiga': 'Manteiga',
    'manteiga#$@!': 'Manteiga',

    'molho de tomate': 'Molho de Tomate',
    'Molho de Tomate': 'Molho de Tomate',
    'mopho de tomate': 'Molho de Tomate',
    'molho de tomate#$@!': 'Molho de Tomate',
    'Molho de Tomate#$@!': 'Molho de Tomate',
    'molmo de tomate': 'Molho de Tomate',

    'refrigerante#$@!': 'Refrigerante',
    'refrigkrante': 'Refrigerante',
    'cerveja#$@!': 'Cerveja',

    'Biscoito Recheado': 'Biscoito Recheado',
    'biscoito recheado#$@!': 'Biscoito Recheado',
    'Biscoitq Recheado': 'Biscoito Recheado',

    'Óleo de Soja': 'Óleo de Soja',
    'óleo de soja#$@!': 'Óleo de Soja',

    'Farinha de Trigo': 'Farinha de Trigo',
    'farinha de trigo#$@!': 'Farinha de Trigo',
    'farinha de tripo': 'Farinha de Trigo',

    'papel higiênico#$@!': 'Papel Higiênico',
    'carvão#$@!': 'Carvão',
    'tal': 'Talco'
}
dict_produtos_corrigidos = {k.lower(): v for k, v in dict_produtos_corrigidos.items()}
df['produto'] = df['produto'].str.lower()
df['produto'] = df['produto'].replace(dict_produtos_corrigidos)
df['produto'] = df['produto'].str.title()
print("🔍 Produtos únicos após correção:")
print(df['produto'].value_counts())

🔍 Produtos únicos após correção:
produto
Pasta De Dente       26250
Queijo Mussarela     26161
Sabonete             24764
Manteiga             24224
Café                 22372
Açúcar               22150
Papel Toalha         18742
Desinfetante         18544
Molho De Tomate      12546
Condicionador        11423
Refrigerante          9643
Água Mineral          9609
Farinha De Trigo      9537
Macarrão              9529
Sal                   9497
Biscoito Recheado     9486
Presunto              9484
Vinho                 9472
Detergente            9458
Shampoo               9443
Óleo De Soja          9389
Suco De Laranja       9261
Cerveja               8626
Pão De Forma          6799
Feijão                6317
Arroz                 6279
Papel Higiênico       5463
Amaciante             4972
Leite Integral        4730
Sabão Em Pó           2998
Carvão                1583
Talco                    1
Name: count, dtype: int64


# 🔍 Análise e Debug da Coluna `vendedores`

Nesta célula, vamos realizar uma análise completa da coluna `vendedores` para identificar:

1. **Valores nulos** (`NaN`)
2. **Strings vazias** ou compostas apenas por espaços
3. **Tipos de dados inconsistentes**
4. **Padrões inválidos** (por exemplo, entradas numéricas, símbolos ou formatação incorreta)
5. **Valores únicos** para inspeção manual

Essas verificações ajudam a garantir a qualidade dos dados antes de qualquer análise ou uso da coluna em modelos.


In [61]:
# 🔍 Debug da coluna 'vendedor'

# 1. Verificar quantidade de valores nulos
nulos = df['vendedor'].isnull().sum()
print(f'🟨 Valores nulos na coluna "vendedor": {nulos}')

# 2. Visualizar linhas com valores nulos
print("\n🧾 Linhas com valores nulos:")
display(df[df['vendedor'].isnull()])

# 3. Verificar valores que são strings vazias ou só espaços
espacos_vazios = df['vendedor'].astype(str).str.strip() == ''
print(f'\n⚠️ Linhas com string vazia ou espaços em branco: {espacos_vazios.sum()}')
display(df[espacos_vazios])

# 4. Verificar tipos de dados na coluna
print("\n🧬 Tipos de dados encontrados na coluna:")
print(df['vendedor'].apply(type).value_counts())

# 5. Verificar padrões inválidos (exemplo: algo que não seja apenas letras e espaços)
padrao_valido = df['vendedor'].astype(str).str.match(r'^[A-Za-zÀ-ÿ\s]+$')
valores_invalidos = df[~padrao_valido]
print(f'\n🚫 Valores com padrão inválido: {valores_invalidos.shape[0]}')
display(valores_invalidos)

# 6. Exibir valores únicos (para inspeção manual)
print("\n🔎 Valores únicos na coluna")
print(df['vendedor'].unique())


🟨 Valores nulos na coluna "vendedor": 0

🧾 Linhas com valores nulos:


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca



⚠️ Linhas com string vazia ou espaços em branco: 0


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca



🧬 Tipos de dados encontrados na coluna:
vendedor
<class 'str'>    368752
Name: count, dtype: int64

🚫 Valores com padrão inválido: 0


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca



🔎 Valores únicos na coluna
['SAMUEL HENRIQUE CAÇADOR' 'MICAEL MALAQUIAS DE SOUZA OLIVEIRA'
 'HENRICO MATOS LIMA DA CUNHA' 'GABRIEL QUEIROZ DE AGUIAR'
 'VICTOR GONÇALVES DONADONI' 'FELIPE HENRIQUE COSTA BARNABE MARAZO'
 'HENRICO VICTOR ALVES' 'CARLOS QUEIROZ DE AGUIAR'
 'LUCAS VITOR FAÇANHA NEVES' 'PAULO SOUZA RONCETE' 'nan']


## ✅ Conclusão da Análise da Coluna `vendedor`

Bom, como não encontramos erros estruturais na coluna `vendedor`, aqui está o resumo:

- 🟨 **Valores nulos reais** (`NaN`): 0
- ⚠️ **Strings vazias ou compostas apenas por espaços**: 0
- 🚫 **Padrões inválidos (ex: números ou símbolos)**: 0
- 🧬 **Todos os dados são do tipo `str`**
- 🔎 **Valores únicos verificados mostram um caso suspeito: `'nan'` (string)**

Embora não seja um `NaN` real, o valor `'nan'` é semântico e logicamente inválido, provavelmente resultado de uma conversão ou importação mal interpretada.

A seguir, vamos tratar esse caso específico substituindo `'nan'` (como string) por `np.nan`, para ser tratado corretamente em operações futuras.


In [62]:
import numpy as np

# 🎯 Substituir valores 'nan' (como string) por np.nan
df['vendedor'] = df['vendedor'].replace('nan', np.nan)

# Verificar novamente após substituição
nulos_corrigidos = df['vendedor'].isnull().sum()
print(f'✅ Após correção, valores nulos na coluna "vendedor": {nulos_corrigidos}')


✅ Após correção, valores nulos na coluna "vendedor": 3680


## 💡 Exemplo de lógica para preenchimento:
* Ideia 1 : Se o mesmo cliente comprou o mesmo produto na mesma data, e em outra linha o vendedor está preenchido, podemos usar esse dado.

* Ideia 2: Se uma combinação de produto, marca e cidade sempre aparece com o mesmo vendedor, podemos assumir esse vendedor para os nulos.

In [63]:
# Primeiro, vamos ver quantas linhas têm vendedor == NaN (após a limpeza anterior)
faltando = df[df['vendedor'].isna()]
print(f"🔎 Linhas com vendedor faltando: {faltando.shape[0]}")

# Exemplo: tentar preencher usando cliente + produto como chave
# Criamos um mapeamento de pares cliente-produto com vendedor conhecido
mapeamento = (
    df[df['vendedor'].notna()]
    .groupby(['cliente', 'produto'])['vendedor']
    .agg(lambda x: x.mode()[0] if not x.mode().empty else np.nan)
)

# Aplicar esse mapeamento para preencher os nulos
def preencher_vendedor(row):
    if pd.isna(row['vendedor']):
        return mapeamento.get((row['cliente'], row['produto']), np.nan)
    return row['vendedor']

df['vendedor'] = df.apply(preencher_vendedor, axis=1)

# Verificar quantos ainda ficaram sem valor
faltando_apos = df['vendedor'].isnull().sum()
print(f"✅ Linhas com vendedor ainda faltando após tentativa de preenchimento: {faltando_apos}")


🔎 Linhas com vendedor faltando: 3680
✅ Linhas com vendedor ainda faltando após tentativa de preenchimento: 4


In [64]:
# Remover as linhas com vendedor ainda faltando
df = df[df['vendedor'].notna()].reset_index(drop=True)

print(f"✅ Base final sem valores nulos em 'vendedor': {df.shape[0]} linhas")


✅ Base final sem valores nulos em 'vendedor': 368748 linhas


In [66]:
# Normalizando o texto da coluna 'produto' para evitar inconsistências
df['produto'] = df['produto'].str.strip().str.lower()

# Removendo o item 'talco' da análise
if (df['produto'] == 'talco').sum() == 1:
    df = df[df['produto'] != 'talco']
    print("✅ Produto 'talco' removido com sucesso.")
else:
    print("⚠️ Produto 'talco' aparece mais de uma vez.")

✅ Produto 'talco' removido com sucesso.


In [67]:
df.to_csv('vendas_limpo.csv', index=False)
print("✅ Arquivo salvo como 'vendas_limpos.csv'")


✅ Arquivo salvo como 'vendas_limpos.csv'


In [69]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth, association_rules

# 🧹 Garante que está trabalhando com uma cópia segura do DataFrame
df = df.copy()

# 🧩 Cria uma nova coluna com Produto + Pagamento
df.loc[:, 'item'] = df['produto'].astype(str).str.strip() + ' | ' + df['pagamento'].astype(str).str.strip()

# 🛒 Agrupa por compra (id_da_compra), gerando listas de itens por transação
transacoes = df.groupby('id_da_compra')['item'].apply(list).tolist()

# 🔄 Transforma as transações para formato binário (necessário para FP-Growth)
te = TransactionEncoder()
df_bin = pd.DataFrame(te.fit(transacoes).transform(transacoes), columns=te.columns_)

# ⛏️ Executa FP-Growth para encontrar conjuntos frequentes
frequentes = fpgrowth(df_bin, min_support=0.02, use_colnames=True)

# 🔗 Gera regras de associação com confiança mínima de 50%
regras = association_rules(frequentes, metric="confidence", min_threshold=0.5)

# 🔍 Exibe as 10 regras com maior lift (maior poder de associação)
regras_ordenadas = regras[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by='lift', ascending=False)

# 👁️ Mostra os resultados
print(regras_ordenadas.head(10))


                                   antecedents  \
10     (refrigerante | transferência bancária)   
11  (suco de laranja | transferência bancária)   
7                         (refrigerante | pix)   
8                      (suco de laranja | pix)   
13          (desinfetante | cartão de crédito)   
12          (papel toalha | cartão de crédito)   
31                        (desinfetante | pix)   
30                        (papel toalha | pix)   
33     (papel toalha | transferência bancária)   
32     (desinfetante | transferência bancária)   

                                   consequents   support  confidence  \
10  (suco de laranja | transferência bancária)  0.020683    0.552538   
11     (refrigerante | transferência bancária)  0.020683    0.570575   
7                      (suco de laranja | pix)  0.021567    0.560659   
8                         (refrigerante | pix)  0.021567    0.575111   
13          (papel toalha | cartão de crédito)  0.039817    0.625065   
12          (desi